In [18]:
import pandas as pd
import folium

In [21]:
data = pd.read_csv('./data/COVID_Data_Basic.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54522 entries, 0 to 54521
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    54522 non-null  int64 
 1   Country       54522 non-null  object
 2   Date          54522 non-null  object
 3   Confirmed     54522 non-null  int64 
 4   Death         54522 non-null  int64 
 5   Recovered     54522 non-null  int64 
 6   newConfirmed  54522 non-null  int64 
 7   newDeath      54522 non-null  int64 
 8   newRecovered  54522 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 3.7+ MB


In [22]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54522 entries, 0 to 54521
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    54522 non-null  int64         
 1   Country       54522 non-null  object        
 2   Date          54522 non-null  datetime64[ns]
 3   Confirmed     54522 non-null  int64         
 4   Death         54522 non-null  int64         
 5   Recovered     54522 non-null  int64         
 6   newConfirmed  54522 non-null  int64         
 7   newDeath      54522 non-null  int64         
 8   newRecovered  54522 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 3.7+ MB


In [23]:
data

,Unnamed: 0,Country,Date,Confirmed,Death,Recovered,newConfirmed,newDeath,newRecovered
0,1,Afghanistan,2019-12-31,0,0,0,0,0,0
1,2,Afghanistan,2020-01-01,0,0,0,0,0,0
2,3,Afghanistan,2020-01-02,0,0,0,0,0,0
3,4,Afghanistan,2020-01-03,0,0,0,0,0,0
4,5,Afghanistan,2020-01-04,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
54517,55096,Zimbabwe,2020-11-02,8389,245,7939,15,2,12
54518,55097,Zimbabwe,2020-11-03,8410,246,7942,21,1,3
54519,55098,Zimbabwe,2020-11-04,8427,248,7967,17,2,25
54520,55099,Zimbabwe,2020-11-05,8444,248,7975,17,0,8


In [24]:
latestDF = data[data['Date'] == max(data['Date'])]

In [25]:
latestDF.reset_index()

,index,Unnamed: 0,Country,Date,Confirmed,Death,Recovered,newConfirmed,newDeath,newRecovered
0,10673,29010,Afghanistan,2020-11-06,41975,1554,34440,40,0,0
1,10905,58010,Albania,2020-11-06,23210,549,11861,489,6,165
2,11137,87010,Algeria,2020-11-06,60800,2024,41510,631,13,266
3,11369,11601,Andorra,2020-11-06,5135,75,3858,0,0,0
4,11601,14501,Angola,2020-11-06,12223,300,5626,121,1,276
...,...,...,...,...,...,...,...,...,...,...
185,53593,53940,West Bank and Gaza,2020-11-06,57226,511,49537,554,3,857
186,53825,54230,Western Sahara,2020-11-06,10,1,8,0,0,0
187,54057,54520,Yemen,2020-11-06,2067,602,1375,4,1,0
188,54289,54810,Zambia,2020-11-06,16819,349,15862,49,0,35


In [26]:
# 파일이 없을때만 크롤링

from selenium import webdriver
import time
import re

# headless
options = webdriver.ChromeOptions()
options.add_argument('headless')

# 위경도 데이터 구하기

latestDF['lat'] = 0
latestDF['lng'] = 0

p = re.compile('@[-0-9]{1,3}.[0-9]*,[-0-9.]*')

for n in latestDF.index:
    print(latestDF['lat'][n])
    url = f'https://www.google.com/maps/place/{latestDF["Country"][n]}'
    
    driver = webdriver.Chrome('chromedriver', chrome_options=options)
    driver.get(url)
    time.sleep(5)

    # 매치된 좌표값 분해
    matchObj = p.search(driver.current_url)
    matchVal = matchObj.group()
    matchGeo = matchVal.split(',')
    
    # @ 제외
    latestDF['lat'][n] = matchGeo[0][1:]
    latestDF['lng'][n] = matchGeo[1]

    print(latestDF['lat'][n], latestDF['lng'][n])
    driver.close()

<ipython-input-26-f860adecae67>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latestDF['lat'] = 0
<ipython-input-26-f860adecae67>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latestDF['lng'] = 0
0
<ipython-input-26-f860adecae67>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latestDF['lat'][n] = matchGeo[0][1:]
C

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=89.0.4389.90)


In [12]:
# 지도에 전세계 확진자 수 표시
m = folium.Map(location=[0, 0], zoom_start=2)

for n in latestDF.index:
    folium.CircleMarker([latestDF['lat'][n], latestDF['lng'][n]], radius=int(latestDF['Confirmed'][n]/75000),
    fill=True, fill_color="red",
    popup= latestDF['Country'][n] + ": " + str(latestDF['Confirmed'][n])
    ).add_to(m)

m


In [ ]:
latestDF.to_csv('./data/COVID_Data_crawling.csv')